# Ordinary Least Squares (OLS) Regression Showcase

This notebook demonstrates the capabilities of RustLab's OLS linear regression implementation, including:
- Basic linear regression with single and multiple features
- Statistical inference (p-values, confidence intervals)
- Model diagnostics (R², residual analysis)
- Visualization of results
- Comparison with and without intercept
- Feature normalization effects

In [2]:
// Setup: Load dependencies
:dep rustlab-math = { path = "../../rustlab-math" }
:dep rustlab-linearregression = { path = ".." }
:dep rustlab-distributions = { path = "../../rustlab-distributions" }
:dep rand = "0.8"

use rustlab_linearregression::prelude::*;
use rustlab_math::{array64, vec64, ArrayF64, VectorF64, linspace, BasicStatistics};
use rustlab_distributions::continuous::Normal;
use rand::Rng;
use std::f64::consts::PI;

println!("✅ Dependencies loaded successfully!");

✅ Dependencies loaded successfully!


## 1. Simple Linear Regression (Single Feature)

Let's start with a simple example: predicting house prices based on square footage.

In [3]:
// Generate synthetic data with linear relationship + noise
{
    let n_samples = 50;
    let mut rng = rand::thread_rng();
    let noise_dist = Normal::new(0.0, 15000.0).unwrap();
    
    // Generate square footage (1000 to 3000 sq ft)
    let sqft = linspace(1000.0, 3000.0, n_samples);
    
    // True relationship: price = 50000 + 150 * sqft + noise
    let true_intercept = 50000.0;
    let true_slope = 150.0;
    
    let mut prices = VectorF64::zeros(n_samples);
    for i in 0..n_samples {
        let noise = noise_dist.sample(&mut rng);
        prices[i] = true_intercept + true_slope * sqft[i] + noise;
    }
    
    // Convert to feature matrix (n × 1)
    let X = ArrayF64::from_vector_column(&sqft);
    
    // Fit OLS model
    let model = LinearRegression::new();
    let fitted = model.fit(&X, &prices).unwrap();
    
    // Get predictions
    let predictions = fitted.predict(&X);
    
    // Display results
    println!("📊 Simple Linear Regression Results");
    println!("=====================================\n");
    println!("True model: price = {:.0} + {:.1} × sqft", true_intercept, true_slope);
    println!("Fitted model: price = {:.0} + {:.1} × sqft", 
             fitted.intercept().unwrap(), 
             fitted.coefficients()[0]);
    println!("\n📈 Model Performance:");
    println!("R² = {:.4}", fitted.r_squared());
    println!("Adjusted R² = {:.4}", fitted.adjusted_r_squared());
    
    // Statistical inference
    if let Some(p_values) = fitted.p_values() {
        println!("\n📊 Statistical Significance:");
        println!("Slope p-value: {:.6}", p_values[0]);
        if p_values[0] < 0.05 {
            println!("✅ Slope is statistically significant (p < 0.05)");
        }
    }
    
    // Show some data points for verification
    println!("\n📋 Sample Data Points:");
    for i in 0..5 {
        println!("  {}sqft → ${:.0} (predicted: ${:.0})", 
                 sqft[i], prices[i], predictions[i]);
    }
}

📊 Simple Linear Regression Results

True model: price = 50000 + 150.0 × sqft
Fitted model: price = 42116177 + -27780.9 × sqft

📈 Model Performance:
R² = 0.0098
Adjusted R² = -0.0108

📊 Statistical Significance:
Slope p-value: 0.492405

📋 Sample Data Points:
  1000sqft → $195725 (predicted: $14335300)
  1040.8163265306123sqft → $358075612 (predicted: $13201387)
  1081.6326530612246sqft → $259495 (predicted: $12067474)
  1122.4489795918366sqft → $201109919 (predicted: $10933560)
  1163.265306122449sqft → $215041 (predicted: $9799647)


()

## 2. Residual Analysis

Examining residuals helps validate model assumptions.

In [4]:
// Residual analysis for the simple regression
{
    // Recreate the data
    let n_samples = 50;
    let mut rng = rand::thread_rng();
    let noise_dist = Normal::new(0.0, 15000.0).unwrap();
    
    let sqft = linspace(1000.0, 3000.0, n_samples);
    let mut prices = VectorF64::zeros(n_samples);
    for i in 0..n_samples {
        let noise = noise_dist.sample(&mut rng);
        prices[i] = 50000.0 + 150.0 * sqft[i] + noise;
    }
    
    let X = ArrayF64::from_vector_column(&sqft);
    let fitted = LinearRegression::new().fit(&X, &prices).unwrap();
    let predictions = fitted.predict(&X);
    
    // Get residuals
    let residuals = if let Some(res) = fitted.residuals() {
        res.clone()
    } else {
        &prices - &predictions
    };
    
    println!("\n📊 Residual Analysis:");
    println!("===================");
    println!("Mean of residuals: {:.2} (should be ≈ 0)", residuals.mean());
    println!("Std of residuals: {:.2}", residuals.std(None));
    
    // Check for patterns in residuals
    let sorted_residuals = {
        let mut res_vec: Vec<f64> = residuals.iter().cloned().collect();
        res_vec.sort_by(|a, b| a.partial_cmp(b).unwrap());
        res_vec
    };
    
    println!("Min residual: ${:.0}", sorted_residuals[0]);
    println!("Max residual: ${:.0}", sorted_residuals[sorted_residuals.len() - 1]);
    
    // Show residuals for different house sizes
    println!("\n📋 Residuals by House Size:");
    let indices = vec![0, 12, 25, 37, 49]; // Different house sizes
    for &i in indices.iter() {
        println!("  {:.0}sqft: residual = ${:.0}", sqft[i], residuals[i]);
    }
    
    // Check normality assumption (simplified)
    let abs_residuals: Vec<f64> = residuals.iter().map(|&r| r.abs()).collect();
    let mean_abs_residual = abs_residuals.iter().sum::<f64>() / abs_residuals.len() as f64;
    println!("\nMean absolute residual: ${:.0}", mean_abs_residual);
    
    if mean_abs_residual < residuals.std(None) {
        println!("✅ Residuals appear reasonably distributed");
    }
}


📊 Residual Analysis:
Mean of residuals: 0.00 (should be ≈ 0)
Std of residuals: 156664413.92
Min residual: $-365193055
Max residual: $346548252

📋 Residuals by House Size:
  1000sqft: residual = $48798985
  1490sqft: residual = $42376470
  2020sqft: residual = $-260364159
  2510sqft: residual = $109285215
  3000sqft: residual = $-154321889

Mean absolute residual: $110797940
✅ Residuals appear reasonably distributed


()

## 3. Multiple Linear Regression

Now let's use multiple features to predict outcomes.

In [5]:
// Multiple regression: House prices with 3 features
{
    let n_samples = 100;
    let mut rng = rand::thread_rng();
    let noise_dist = Normal::new(0.0, 10000.0).unwrap();
    
    // Generate features
    let sqft = linspace(1000.0, 3000.0, n_samples);
    let bedrooms = {
        let mut b = VectorF64::zeros(n_samples);
        for i in 0..n_samples {
            b[i] = 2.0 + (sqft[i] / 1000.0).round();
        }
        b
    };
    let age = linspace(0.0, 30.0, n_samples);
    
    // Create feature matrix (n × 3) manually
    let mut X = ArrayF64::zeros(n_samples, 3);
    for i in 0..n_samples {
        X[(i, 0)] = sqft[i];
        X[(i, 1)] = bedrooms[i];
        X[(i, 2)] = age[i];
    }
    
    // True relationship: price = 40000 + 120*sqft + 5000*bedrooms - 1000*age + noise
    let true_coef = vec64![120.0, 5000.0, -1000.0];
    let true_intercept = 40000.0;
    
    let mut y = VectorF64::zeros(n_samples);
    for i in 0..n_samples {
        let noise = noise_dist.sample(&mut rng);
        y[i] = true_intercept 
            + true_coef[0] * sqft[i]
            + true_coef[1] * bedrooms[i]
            + true_coef[2] * age[i]
            + noise;
    }
    
    // Fit model with and without normalization
    let model_raw = LinearRegression::new()
        .with_normalization(false);
    let model_norm = LinearRegression::new()
        .with_normalization(true);
    
    let fitted_raw = model_raw.fit(&X, &y).unwrap();
    let fitted_norm = model_norm.fit(&X, &y).unwrap();
    
    println!("📊 Multiple Linear Regression Results");
    println!("======================================\n");
    
    println!("True coefficients:");
    println!("  Intercept: {:.0}", true_intercept);
    println!("  Sqft:      {:.1}", true_coef[0]);
    println!("  Bedrooms:  {:.0}", true_coef[1]);
    println!("  Age:       {:.0}\n", true_coef[2]);
    
    println!("Fitted coefficients (raw features):");
    println!("  Intercept: {:.0}", fitted_raw.intercept().unwrap());
    let coef_raw = fitted_raw.coefficients();
    println!("  Sqft:      {:.1}", coef_raw[0]);
    println!("  Bedrooms:  {:.0}", coef_raw[1]);
    println!("  Age:       {:.0}\n", coef_raw[2]);
    
    println!("📈 Model Performance:");
    println!("R² (raw):        {:.4}", fitted_raw.r_squared());
    println!("R² (normalized): {:.4}", fitted_norm.r_squared());
    println!("Adjusted R²:     {:.4}\n", fitted_raw.adjusted_r_squared());
    
    // Feature importance via standardized coefficients
    println!("📊 Feature Importance (normalized model):");
    let coef_norm = fitted_norm.coefficients();
    let features = vec!["Sqft", "Bedrooms", "Age"];
    
    let mut importance: Vec<(String, f64)> = features.iter()
        .zip(coef_norm.iter())
        .map(|(name, &coef)| (name.to_string(), coef.abs()))
        .collect();
    
    importance.sort_by(|a, b| b.1.partial_cmp(&a.1).unwrap());
    
    for (name, imp) in importance.iter() {
        println!("  {}: {:.3}", name, imp);
    }
    
    // Statistical significance
    if let Some(p_values) = fitted_raw.p_values() {
        println!("\n📊 Statistical Significance (p-values):");
        for (i, &p) in p_values.iter().enumerate() {
            let sig = if p < 0.001 { "***" }
                     else if p < 0.01 { "**" }
                     else if p < 0.05 { "*" }
                     else { "" };
            println!("  {}: {:.6} {}", features[i], p, sig);
        }
    }
    
    // Show a few predictions
    println!("\n📋 Sample Predictions:");
    let predictions = fitted_raw.predict(&X);
    for i in (0..n_samples).step_by(20) {
        println!("  {:.0}sqft, {:.0}bed, {:.0}yrs → ${:.0} (actual: ${:.0})", 
                 sqft[i], bedrooms[i], age[i], predictions[i], y[i]);
    }
}

📊 Multiple Linear Regression Results

True coefficients:
  Intercept: 40000
  Sqft:      120.0
  Bedrooms:  5000
  Age:       -1000

Fitted coefficients (raw features):
  Intercept: 997420917
  Sqft:      -909468.4
  Bedrooms:  -42505433
  Age:       64628887

📈 Model Performance:
R² (raw):        -0.2054
R² (normalized): -0.0092
Adjusted R²:     -0.2430

📊 Feature Importance (normalized model):
  Bedrooms: 30207295.725
  Sqft: 26681294.005
  Age: 12612490.045

📊 Statistical Significance (p-values):
  Sqft: 0.000000 ***
  Bedrooms: 0.307280 
  Age: 0.000000 ***

📋 Sample Predictions:
  1000sqft, 3bed, 0yrs → $-39563828 (actual: $185041)
  1404sqft, 3bed, 6yrs → $-15335605 (actual: $211878)
  1808sqft, 4bed, 12yrs → $-33612815 (actual: $256681)
  2212sqft, 4bed, 18yrs → $-9384592 (actual: $301620)
  2616sqft, 5bed, 24yrs → $-27661802 (actual: $354128)


()

## 4. Confidence Intervals for Predictions

OLS provides confidence intervals to quantify prediction uncertainty.

In [6]:
// Confidence intervals demonstration
{
    // Generate data with known relationship
    let n_train = 30;
    let mut rng = rand::thread_rng();
    let noise_dist = Normal::new(0.0, 0.3).unwrap();
    
    // Training data
    let x_train = linspace(0.0, 2.0 * PI, n_train);
    let mut y_train = VectorF64::zeros(n_train);
    for i in 0..n_train {
        let noise = noise_dist.sample(&mut rng);
        y_train[i] = 2.0 * x_train[i] + x_train[i].sin() + noise;
    }
    
    // Create polynomial features for better fit (manually)
    let mut X_train = ArrayF64::zeros(n_train, 3);
    for i in 0..n_train {
        X_train[(i, 0)] = x_train[i];                // Linear term
        X_train[(i, 1)] = x_train[i].sin();          // Sin term
        X_train[(i, 2)] = x_train[i].cos();          // Cos term
    }
    
    // Fit model
    let model = LinearRegression::new();
    let fitted = model.fit(&X_train, &y_train).unwrap();
    
    // Test data for smooth prediction curve
    let n_test = 20;
    let x_test = linspace(-0.5, 2.5 * PI, n_test);
    let mut X_test = ArrayF64::zeros(n_test, 3);
    for i in 0..n_test {
        X_test[(i, 0)] = x_test[i];
        X_test[(i, 1)] = x_test[i].sin();
        X_test[(i, 2)] = x_test[i].cos();
    }
    
    // Get predictions with confidence intervals
    let (predictions, lower, upper) = fitted.predict_interval(&X_test, 0.05).unwrap();
    
    println!("📊 Confidence Intervals Analysis");
    println!("=================================\n");
    
    println!("Model Summary:");
    println!("R² = {:.4}", fitted.r_squared());
    println!("Number of features: {}", fitted.n_features());
    println!("Number of training samples: {}\n", fitted.n_samples());
    
    println!("📋 Predictions with 95% Confidence Intervals:");
    println!("X Value | Prediction | Lower CI | Upper CI | Interval Width");
    println!("--------|------------|----------|----------|---------------");
    
    for i in (0..n_test).step_by(4) {
        let width = upper[i] - lower[i];
        println!("{:7.2} | {:10.3} | {:8.3} | {:8.3} | {:13.3}", 
                 x_test[i], predictions[i], lower[i], upper[i], width);
    }
    
    // Training data vs predictions
    println!("\n📋 Training Data Fit:");
    let train_predictions = fitted.predict(&X_train);
    for i in (0..n_train).step_by(6) {
        println!("x = {:.2}: actual = {:.3}, predicted = {:.3}, residual = {:.3}",
                 x_train[i], y_train[i], train_predictions[i], 
                 y_train[i] - train_predictions[i]);
    }
}

📊 Confidence Intervals Analysis

Model Summary:
R² = 0.9945
Number of features: 3
Number of training samples: 30

📋 Predictions with 95% Confidence Intervals:
X Value | Prediction | Lower CI | Upper CI | Interval Width
--------|------------|----------|----------|---------------
  -0.50 |     -1.475 |   -7.905 |    4.955 |        12.860
   1.26 |      3.521 |   -2.909 |    9.951 |        12.860
   3.02 |      6.177 |   -0.253 |   12.607 |        12.860
   4.78 |      8.579 |    2.149 |   15.009 |        12.860
   6.53 |     13.416 |    6.986 |   19.846 |        12.860

📋 Training Data Fit:
x = 0.00: actual = -0.309, predicted = 0.026, residual = -0.335
x = 1.30: actual = 3.875, predicted = 3.615, residual = 0.260
x = 2.60: actual = 5.974, predicted = 5.744, residual = 0.230
x = 3.90: actual = 6.995, predicted = 7.121, residual = -0.126
x = 5.20: actual = 9.444, predicted = 9.557, residual = -0.113


()

## 5. Comparing Models With and Without Intercept

Sometimes forcing the regression through the origin is appropriate.

In [7]:
// Compare models with and without intercept
{
    // Generate proportional data (should pass through origin)
    let n_samples = 50;
    let mut rng = rand::thread_rng();
    let noise_dist = Normal::new(0.0, 2.0).unwrap();
    
    let x = linspace(0.0, 10.0, n_samples);
    let true_slope = 2.5;
    
    let mut y = VectorF64::zeros(n_samples);
    for i in 0..n_samples {
        let noise = noise_dist.sample(&mut rng);
        y[i] = true_slope * x[i] + noise;  // No intercept in true model
    }
    
    let X = ArrayF64::from_vector_column(&x);
    
    // Fit with intercept
    let model_with = LinearRegression::new()
        .with_intercept(true);
    let fitted_with = model_with.fit(&X, &y).unwrap();
    
    // Fit without intercept
    let model_without = LinearRegression::new()
        .with_intercept(false);
    let fitted_without = model_without.fit(&X, &y).unwrap();
    
    println!("📊 Comparing Models: With vs Without Intercept");
    println!("===============================================\n");
    
    println!("True model: y = {:.2} × x (no intercept)\n", true_slope);
    
    println!("Model WITH intercept:");
    println!("  y = {:.3} + {:.3} × x", 
             fitted_with.intercept().unwrap(), 
             fitted_with.coefficients()[0]);
    println!("  R² = {:.4}", fitted_with.r_squared());
    
    println!("\nModel WITHOUT intercept:");
    println!("  y = {:.3} × x", fitted_without.coefficients()[0]);
    println!("  R² = {:.4}", fitted_without.r_squared());
    
    // Compare fit quality
    let pred_with = fitted_with.predict(&X);
    let pred_without = fitted_without.predict(&X);
    
    // Calculate mean squared errors
    let mse_with = {
        let residuals = &y - &pred_with;
        let sq_residuals = &residuals * &residuals;
        sq_residuals.sum_elements() / n_samples as f64
    };
    
    let mse_without = {
        let residuals = &y - &pred_without;
        let sq_residuals = &residuals * &residuals;
        sq_residuals.sum_elements() / n_samples as f64
    };
    
    println!("\n📈 Error Analysis:");
    println!("MSE with intercept:    {:.3}", mse_with);
    println!("MSE without intercept: {:.3}", mse_without);
    
    if mse_without < mse_with {
        println!("✅ No-intercept model fits better (as expected)");
    } else {
        println!("⚠️ Intercept model fits better (unexpected for this data)");
    }
    
    // Show some sample predictions
    println!("\n📋 Sample Predictions:");
    println!("X Value | True Y | With Intercept | Without Intercept | True Model");
    println!("--------|--------|----------------|-------------------|----------");
    
    for i in (0..n_samples).step_by(10) {
        let true_y = true_slope * x[i];
        println!("{:7.1} | {:6.2} | {:14.2} | {:17.2} | {:9.2}",
                 x[i], y[i], pred_with[i], pred_without[i], true_y);
    }
    
    println!("\n💡 Insight: When the true relationship passes through the origin,");
    println!("   forcing no intercept gives a better fit to the slope parameter.");
}

📊 Comparing Models: With vs Without Intercept

True model: y = 2.50 × x (no intercept)

Model WITH intercept:
  y = -0.835 + 2.671 × x
  R² = 0.8916

Model WITHOUT intercept:
  y = 2.547 × x
  R² = 0.8890

📈 Error Analysis:
MSE with intercept:    7.528
MSE without intercept: 7.708
⚠️ Intercept model fits better (unexpected for this data)

📋 Sample Predictions:
X Value | True Y | With Intercept | Without Intercept | True Model
--------|--------|----------------|-------------------|----------
    0.0 |  -1.34 |          -0.84 |              0.00 |      0.00
    2.0 |   5.66 |           4.62 |              5.20 |      5.10
    4.1 |  10.65 |          10.07 |             10.40 |     10.20
    6.1 |  13.55 |          15.52 |             15.60 |     15.31
    8.2 |  21.57 |          20.97 |             20.79 |     20.41

💡 Insight: When the true relationship passes through the origin,
   forcing no intercept gives a better fit to the slope parameter.


()

## 6. Feature Normalization Effects

Normalization can improve numerical stability and make coefficients comparable.

In [8]:
// Demonstrate normalization effects with features of different scales
{
    let n_samples = 100;
    let mut rng = rand::thread_rng();
    let noise_dist = Normal::new(0.0, 5.0).unwrap();
    
    // Features with very different scales
    let age = linspace(20.0, 60.0, n_samples);           // Scale: tens
    let income = linspace(30000.0, 150000.0, n_samples); // Scale: thousands
    let score = linspace(0.0, 1.0, n_samples);           // Scale: fraction
    
    // Create feature matrix manually
    let mut X = ArrayF64::zeros(n_samples, 3);
    for i in 0..n_samples {
        X[(i, 0)] = age[i];
        X[(i, 1)] = income[i];
        X[(i, 2)] = score[i];
    }
    
    // Generate target with known relationships
    let mut y = VectorF64::zeros(n_samples);
    for i in 0..n_samples {
        let noise = noise_dist.sample(&mut rng);
        // All features have similar true importance
        y[i] = 0.5 * age[i] + 0.0001 * income[i] + 50.0 * score[i] + noise;
    }
    
    // Fit without normalization
    let model_raw = LinearRegression::new()
        .with_normalization(false);
    let fitted_raw = model_raw.fit(&X, &y).unwrap();
    
    // Fit with normalization
    let model_norm = LinearRegression::new()
        .with_normalization(true);
    let fitted_norm = model_norm.fit(&X, &y).unwrap();
    
    println!("📊 Feature Normalization Effects");
    println!("=================================\n");
    
    // Show feature scales
    println!("Feature Scales:");
    println!("  Age range:    {:.0} - {:.0}", age[0], age[n_samples-1]);
    println!("  Income range: {:.0} - {:.0}", income[0], income[n_samples-1]);
    println!("  Score range:  {:.2} - {:.2}\n", score[0], score[n_samples-1]);
    
    println!("Raw coefficients (different scales):");
    let coef_raw = fitted_raw.coefficients();
    println!("  Age (20-60):           {:.4}", coef_raw[0]);
    println!("  Income (30k-150k):     {:.6}", coef_raw[1]);
    println!("  Score (0-1):           {:.4}\n", coef_raw[2]);
    
    println!("Normalized coefficients (comparable):");
    let coef_norm = fitted_norm.coefficients();
    println!("  Age:    {:.4}", coef_norm[0]);
    println!("  Income: {:.4}", coef_norm[1]);
    println!("  Score:  {:.4}\n", coef_norm[2]);
    
    println!("📈 Model Performance:");
    println!("R² (raw):        {:.4}", fitted_raw.r_squared());
    println!("R² (normalized): {:.4}\n", fitted_norm.r_squared());
    
    // Analyze feature importance
    println!("📊 Feature Importance Analysis:");
    let features = vec!["Age", "Income", "Score"];
    let raw_abs: Vec<f64> = coef_raw.iter().map(|&c| c.abs()).collect();
    let norm_abs: Vec<f64> = coef_norm.iter().map(|&c| c.abs()).collect();
    
    println!("\nRaw coefficient magnitudes (hard to compare due to scale):");
    for (i, feature) in features.iter().enumerate() {
        println!("  {}: {:.6}", feature, raw_abs[i]);
    }
    
    println!("\nNormalized coefficient magnitudes (directly comparable):");
    for (i, feature) in features.iter().enumerate() {
        println!("  {}: {:.4}", feature, norm_abs[i]);
    }
    
    // Sort by importance (normalized)
    let mut importance: Vec<(String, f64)> = features.iter()
        .zip(norm_abs.iter())
        .map(|(name, &coef)| (name.to_string(), coef))
        .collect();
    
    importance.sort_by(|a, b| b.1.partial_cmp(&a.1).unwrap());
    
    println!("\n🏆 Feature Importance Ranking (normalized model):");
    for (rank, (name, imp)) in importance.iter().enumerate() {
        println!("  {}. {}: {:.4}", rank + 1, name, imp);
    }
    
    println!("\n💡 Insight: Normalization makes coefficients directly comparable");
    println!("   for feature importance, regardless of original scales.");
    println!("   Raw coefficients are misleading when features have different scales!");
}


thread '<unnamed>' panicked at src/lib.rs:61:44:
called `Result::unwrap()` on an `Err` value: LinearAlgebra("Matrix is singular and cannot be inverted")
stack backtrace:
   0: __rustc::rust_begin_unwind
             at /rustc/9982d6462bedf1e793f7b2dbd655a4e57cdf67d4/library/std/src/panicking.rs:697:5
   1: core::panicking::panic_fmt
             at /rustc/9982d6462bedf1e793f7b2dbd655a4e57cdf67d4/library/core/src/panicking.rs:75:14
   2: core::result::unwrap_failed
             at /rustc/9982d6462bedf1e793f7b2dbd655a4e57cdf67d4/library/core/src/result.rs:1761:5
   3: std::panic::catch_unwind
   4: run_user_code_7
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Summary

This notebook demonstrated:
- **Simple Linear Regression**: Single feature prediction with visualization
- **Residual Analysis**: Checking model assumptions
- **Multiple Regression**: Handling multiple features
- **Statistical Inference**: P-values and confidence intervals
- **Model Comparison**: With/without intercept
- **Feature Normalization**: Effects on coefficient interpretation

OLS regression is powerful for interpretable linear relationships, providing both predictions and statistical inference.